# Train CIFAR with the `policy` module

Let's install the latest version of Poutyne (if it's not already) and import all the needed packages.

In [1]:
%pip install --upgrade poutyne

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18

from poutyne import Model, OptimizerPolicy, one_cycle_phases

## Training constant
But first, let's set the training constants, the CUDA device used for training if one is present, we set the batch size (i.e. the number of elements to see before updating the model) and the number of epochs (i.e. the number of times we see the full dataset).

In [2]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 1024
epochs = 5

# Load the data

In [3]:
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

In [4]:
root = "data"
train_ds = datasets.CIFAR10(root, train=True, transform=train_transform, download=True)
val_ds = datasets.CIFAR10(root, train=False, transform=val_transform, download=True)

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [5]:
train_dl = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True
)
val_dl = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False
)

# The model
We'll train a simple `resNet-18` network.
This takes a while without GPU but is pretty quick with GPU.

In [6]:
def get_module():
    model = resnet18(pretrained=False)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc = nn.Linear(512, 10)
    return model


# Training without the `policies` module

In [7]:
pytorch_network = get_module().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
)
model = model.to(device)

history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
)





Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|8.58s loss: 2.115964 acc: 22.938000 val_loss: 1.869150 val_acc: 32.870000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|8.36s loss: 1.805687 acc: 34.560000 val_loss: 1.682006 val_acc: 39.280000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|8.38s loss: 1.658098 acc: 40.008000 val_loss: 1.610190 val_acc: 41.670000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|8.36s loss: 1.563534 acc: 43.264000 val_loss: 1.502107 val_acc: 45.430000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|8.41s loss: 1.504496 acc: 45.646000 val_loss: 1.461815 val_acc: 46.660000


# Training with the `policies` module

In [8]:
steps_per_epoch = len(train_dl)
steps_per_epoch

49

In [9]:
pytorch_network = get_module().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
)
model = model.to(device)

policy = OptimizerPolicy(
    one_cycle_phases(epochs * steps_per_epoch, lr=(0.01, 0.1, 0.008)),
)
history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
    callbacks=[policy],
)

Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|8.50s loss: 1.847516 acc: 33.342000 val_loss: 1.675540 val_acc: 42.400000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|8.56s loss: 1.400317 acc: 49.748000 val_loss: 1.323423 val_acc: 52.840000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|8.53s loss: 1.142495 acc: 59.492000 val_loss: 1.117616 val_acc: 61.920000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|8.53s loss: 0.946500 acc: 66.758000 val_loss: 0.941687 val_acc: 67.420000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|8.51s loss: 0.782936 acc: 72.502000 val_loss: 0.847208 val_acc: 70.250000
